# TensorFlow2 で Amazon SageMaker Experiments を使う

このノートブックでは、optimizer と batci_size を変えて複数の学習ジョブを実行し、それらのジョブを Amazon SageMaker Experiments で管理する方法を説明します。


Amazon SageMaker Experiments SDK のドキュメントは [こちら](https://sagemaker-experiments.readthedocs.io/en/latest/index.html) から参照できます。

# 環境のセットアップ

SageMaker Experiments をインストールします。

In [ ]:
import sys
!{sys.executable} -m pip install sagemaker-experiments

必要なライブラリをインポートします。

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

from sagemaker.analytics import ExperimentAnalytics
from sagemaker.session import Session

import boto3
import time
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

## 学習データ

MNIST データセットは S3 バケット ``sagemaker-sample-data-<REGION>`` の中の ``tensorflow/mnist`` に保存されています。以下の 4 つの``.npy`` ファイルがあります。
* ``train_data.npy``
* ``eval_data.npy``
* ``train_labels.npy``
* ``eval_labels.npy``

In [ ]:
training_data_uri = 's3://sagemaker-sample-data-{}/tensorflow/mnist'.format(region)

## Experiment のセットアップ

ジョブを管理するための Experiment を作成します。

In [ ]:
boto3_sess = boto3.Session()
sm = boto3_sess.client('sagemaker')

tf_experiment = Experiment.create(
    experiment_name=f"tensorflow-mnist-classification-{int(time.time())}", 
    description="Classification of mnist hand-written digits", 
    sagemaker_boto_client=sm)
print(tf_experiment)

# 学習ジョブの実行と Experiment への登録

optimizer と batch_size の選択肢を list として用意し、組み合わせを変えながら学習ジョブを実行します。また、各学習ジョブを Experiment の Trial として登録します。

In [ ]:
optimizers = ['adam', 'sgd']
batch_size_list = [16, 32]

metric_definitions = [{'Name': 'loss',
                       'Regex': 'loss: ([0-9\\.]+)'},
                      {'Name': 'accuracy',
                       'Regex': 'accuracy: ([0-9\\.]+)'},
                     {'Name': 'test loss',
                       'Regex': 'test loss: ([0-9\\.]+)'},
                     {'Name': 'test acc',
                       'Regex': 'test acc: ([0-9\\.]+)'}]

for optimizer in optimizers:
    for batch_size in batch_size_list:

        hyperparameters = {'optimizer': optimizer, 'batch_size': batch_size}
        
        # Trial の作成
        trial_name = f'tf-training-job-{optimizer}-{batch_size}-{int(time.time())}'
        print(trial_name)
        
        tf_trial = Trial.create(
            trial_name=trial_name, 
            experiment_name=tf_experiment.experiment_name,
            sagemaker_boto_client=sm,
        )

        mnist_estimator = TensorFlow(entry_point='mnist.py',
                                     role=role,
                                     train_instance_count=1,
                                     train_instance_type='ml.p3.2xlarge',
                                     framework_version='2.1.0',
                                     py_version='py3',
                                      hyperparameters=hyperparameters,
                                     metric_definitions=metric_definitions,
                                     enable_sagemaker_metrics=True,
                                     distributions={'parameter_server': {'enabled': True}})
        
        tf_training_job_name = "tensorflow-training-job-{}".format(int(time.time()))
        
        mnist_estimator.fit(
                            training_data_uri,
                            job_name=tf_training_job_name,
                            experiment_config={
                                 'TrialName': tf_trial.trial_name,
                                 'TrialComponentDisplayName': 'Training',
                            },
                            wait=False)

 ## Experiments を可視化
 
 すべての Trial を accuracy でソートして表示します。

In [ ]:
search_expression = {
    "Filters":[
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=Session(boto3_sess, sm), 
    experiment_name=tf_experiment.experiment_name,
    search_expression=search_expression,
#     sort_by="metrics.accuracy.max",
    sort_by="metrics.test acc.max",
#     sort_by="CreationTime",
    sort_order="Descending",
#     metric_names=['accuracy'],
#     metric_names=['test acc', 'test loss'],
    parameter_names=['optimizer', 'batch_size']
)

analytic_table = trial_component_analytics.dataframe()

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
analytic_table

# 最も結果が良かったモデルをデプロイ

こちらは、すべての学習ジョブが完了するまで待ってから実行してください。 <br>作成したモデルの中で、最も accuracy が高かったモデルをデプロイします。

In [ ]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data=analytic_table['SageMaker.ModelArtifact - Value'][0],
                                            role=role,
                                            entry_point='mnist.py',
                                            framework_version='2.1.0'
             )

predictor = model.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')

# 推論エンドポイントを使って推論

学習に使用したデータを使って推論します。まずデータを S3 からダウンロードします。

In [ ]:
import numpy as np

!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_data.npy train_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_labels.npy train_labels.npy

train_data = np.load('train_data.npy')
train_labels = np.load('train_labels.npy')

ダウンロードしたデータを使って推論し。その結果を表示します。

In [ ]:
predictions = predictor.predict(train_data[:50])
for i in range(0, 50):
    prediction = np.argmax(predictions['predictions'][i])
    label = train_labels[i]
    print('prediction is {}, label is {}, matched: {}'.format(prediction, label, prediction == label))

# リソースの削除
## エンドポイントの削除

立ち上がりっぱなしだと料金がかかるので、不要になったエンドポイントを削除します。

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)

## Experiment の削除

このノートブックで作成した Experiment を削除します。

In [ ]:
def cleanup_sme_sdk(experiment):
    for trial_summary in experiment.list_trials():
        trial = Trial.load(trial_name=trial_summary.trial_name)
        for trial_component_summary in trial.list_trial_components():
            tc = TrialComponent.load(
                trial_component_name=trial_component_summary.trial_component_name)
            trial.remove_trial_component(tc)
            try:
                # comment out to keep trial components
                tc.delete()
            except:
                # tc is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        trial.delete()
        experiment_name = experiment.experiment_name
    experiment.delete()
    print(f"\nExperiment {experiment_name} deleted")

In [ ]:
experiment_to_cleanup = Experiment.load(
    # Use experiment name not display name
    experiment_name=tf_experiment.experiment_name)

cleanup_sme_sdk(experiment_to_cleanup)

# [option] Experiment 一覧を表示

今までにどんな　Experiments を作成したかを知りたい場合、以下の方法で Experiments の一覧表示が可能です。

In [ ]:
lst = Experiment.list()

In [ ]:
for i, d in enumerate(lst):
    print('experiment_name:', i, d.experiment_name)